# Data Cleaning - Census/FRED

Data cleaning for ECON672 Project
This file aggregates Census Population estimates and 
FRED annual unemployment data.

In [22]:
%%capture
# Capture suppresses output to screen
import stata_setup
stata_setup.config("/Applications/Stata 17", "be")

In [23]:
%%stata
/* Define paths for data and output */
global data "Data"
global output "Output"

global raw "$data/Raw"
global fredUnemploy "FRED/Unemployment"
global pop2000 "US_Census/2000-2010"
global pop2010 "US_Census/2010-2019"

global rawUnemploy "$raw/$fredUnemploy"
global rawPop2000 "$raw/$pop2000"
global rawPop2010 "$raw/$pop2010"

global processed "$data/Processed"
global processedUnemploy "$processed/$fredUnemploy"
global processedPop2000 "$processed/$pop2000"
global processedPop2010 "$processed/$pop2010"

global finalFredData "$processed/FRED/MergedFred.dta"
global mergedPop2000 "$processed/US_Census/allPop2000.dta"
global finalCensusData "$processed/US_Census/MergedCensus.dta"


. /* Define paths for data and output */
. global data "Data"

. global output "Output"

. 
. global raw "$data/Raw"

. global fredUnemploy "FRED/Unemployment"

. global pop2000 "US_Census/2000-2010"

. global pop2010 "US_Census/2010-2019"

. 
. global rawUnemploy "$raw/$fredUnemploy"

. global rawPop2000 "$raw/$pop2000"

. global rawPop2010 "$raw/$pop2010"

. 
. global processed "$data/Processed"

. global processedUnemploy "$processed/$fredUnemploy"

. global processedPop2000 "$processed/$pop2000"

. global processedPop2010 "$processed/$pop2010"

. 
. global finalFredData "$processed/FRED/MergedFred.dta"

. global mergedPop2000 "$processed/US_Census/allPop2000.dta"

. global finalCensusData "$processed/US_Census/MergedCensus.dta"

. 


In [24]:
%%capture
%%stata
log using "$output/dataClean-syth-census-fred.log", replace

## Import FRED Uemploymen data
Data is FRED state-level annual unemployment rate, LAUST - series

| i.year | c.lnUnmploy | i.state |
| --- | ---- | --- |
| year | ln(unEmploy) |  MO |
| year | ln(unEmploy) |  WA |  

## Import  Unemployment Numbers from the States

In [25]:
%%stata
/* Read all the xlsx files into Stata files */   
local fileList: dir "$rawUnemploy" files "*"
local n_files: word count `fileList'
 
forval i=1/`n_files' {
    local thisFile `: word `i' of `fileList''
    scalar thisState = substr("`thisFile'", 1, 2)
    
    di ">>>Importing data for ", thisState, ": `thisFile'"
    import excel "$rawUnemploy/`thisFile'", /// 
    sheet("FRED Graph") /// 
    cellrange(A11:B56) /// 
    firstrow ///
    clear
    
    /* 
        Change series name to useful variable and 
        create ln(unemployment)
    */
    rename LAUST* unemploy
    label variable ///
        unemploy "Annual state level unemployment rate (not seasonaly adjusted)"
    generate lnUnemploy=ln(unemploy)
    label variable ///
        lnUnemploy "ln(unemploy)"
    
    /* Convert Day Month Year time to year */
    generate year=year(observation_date)
    drop observation_date
    
    local dtaFile: di (thisState+".dta")
    save "$processedUnemploy/`dtaFile'", replace
    
}


. /* Read all the xlsx files into Stata files */   
. local fileList: dir "$rawUnemploy" files "*"

. local n_files: word count `fileList'

.  
. forval i=1/`n_files' {
  2.     local thisFile `: word `i' of `fileList''
  3.     scalar thisState = substr("`thisFile'", 1, 2)
  4.     
.     di ">>>Importing data for ", thisState, ": `thisFile'"
  5.     import excel "$rawUnemploy/`thisFile'", /// 
>     sheet("FRED Graph") /// 
>     cellrange(A11:B56) /// 
>     firstrow ///
>     clear
  6.     
.     /* 
>         Change series name to useful variable and 
>         create ln(unemployment)
>     */
.     rename LAUST* unemploy
  7.     label variable ///
>         unemploy "Annual state level unemployment rate (not seasonaly adjuste
> d)"
  8.     generate lnUnemploy=ln(unemploy)
  9.     label variable ///
>         lnUnemploy "ln(unemploy)"
 10.     
.     /* Convert Day Month Year time to year */
.     generate year=year(observation_date)
 11.     drop observation_date
 12.     


In [26]:
%%stata 
/* Append all the State-level Stata files into a single file */   
local fileList: dir "$processedUnemploy" files "*"
local n_files: word count `fileList'
 
forval i=1/`n_files' {
    local thisFile `: word `i' of `fileList''
    scalar thisState = substr("`thisFile'", 1, 2)                           
    

    /* Special actions for first file */
    if `i'==1 {
        di ">>>Importing data for ", thisState, ": `thisFile'"
        use "$processedUnemploy/`thisFile'", clear
        generate state = thisState
    }

    /* Actions for all subsequent files */
    if `i'!=1 {
        di ">>>Importing data for ", thisState, ": `thisFile'"
        append using "$processedUnemploy/`thisFile'"
        replace state = thisState if state==""
    }
    
    if `i'==`n_files'{
        /* Drop obs for years prior to 2001 and after 2020*/
        drop if year<2001
        drop if year>2020
        save "$finalFredData", replace
    }

}


. /* Append all the State-level Stata files into a single file */   
. local fileList: dir "$processedUnemploy" files "*"

. local n_files: word count `fileList'

.  
. forval i=1/`n_files' {
  2.     local thisFile `: word `i' of `fileList''
  3.     scalar thisState = substr("`thisFile'", 1, 2)                         
>   
  4.     
. 
.     /* Special actions for first file */
.     if `i'==1 {
  5.         di ">>>Importing data for ", thisState, ": `thisFile'"
  6.         use "$processedUnemploy/`thisFile'", clear
  7.         generate state = thisState
  8.     }
  9. 
.     /* Actions for all subsequent files */
.     if `i'!=1 {
 10.         di ">>>Importing data for ", thisState, ": `thisFile'"
 11.         append using "$processedUnemploy/`thisFile'"
 12.         replace state = thisState if state==""
 13.     }
 14.     
.     if `i'==`n_files'{
 15.         /* Drop obs for years prior to 2001 and after 2020*/
.         drop if year<2001
 16.         drop if year>2020
 17.

In [27]:
%%stata
describe
summarize
tab state


. describe

Contains data from Data/Processed/FRED/MergedFred.dta
 Observations:           680                  
    Variables:             4                  5 Feb 2022 17:31
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
unemploy        double  %10.0g                Annual state level unemployment
                                                rate (not seasonaly adjusted)
lnUnemploy      float   %9.0g                 ln(unemploy)
year            float   %9.0g                 
state           str2    %9s                   
-------------------------------------------------------------------------------
Sorted by: 

. summarize

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
  

---

---

## US Census Data Population Estimates
There are two main datasets:
+ 2000-2010 is contained in a single file per state 
+ 2010-2019 is contained in a single file that includes all the states  

### US Census 2000-2010

In [28]:
%%stata
/* Read all the xlsx files into Stata files */   
local fileList: dir "$rawPop2000" files "*"
local n_files: word count `fileList'
 
forval i=1/`n_files' {
    local thisFile `: word `i' of `fileList''
    scalar thisState = substr("`thisFile'", 1, 2)
    
    di ">>>Importing data for ", thisState, ": `thisFile'"
    import excel "$rawPop2000/`thisFile'", /// 
    sheet("ST-EST00INT-02") /// 
    cellrange(A4:N5) /// 
    firstrow ///
    clear
    
    /*  Drop columns */
    drop A B M N
    
    /* State ID */
    generate state = thisState
    
    /* Rename columns to prep for reshape */
    scalar yearCount = 2000
        foreach var of varlist C-L{
            local j: display int(yearCount)
            rename `var' pop`j'
            scalar yearCount = yearCount + 1
        }
    
    /* 
        Reshape the data to a long format
    */
    reshape long pop, i(state) j(year) 
    
    /* 
    Convert to Strings by ignoring the ",".  
      Raw data is formated with comma separator for thousands
      and millions.
    */
    destring(pop), replace ignore(",")
    label variable pop "US Census Population Estimate"
    
    
    local dtaFile: di (thisState+".dta")
    save "$processedPop2000/`dtaFile'", replace
    
}


. /* Read all the xlsx files into Stata files */   
. local fileList: dir "$rawPop2000" files "*"

. local n_files: word count `fileList'

.  
. forval i=1/`n_files' {
  2.     local thisFile `: word `i' of `fileList''
  3.     scalar thisState = substr("`thisFile'", 1, 2)
  4.     
.     di ">>>Importing data for ", thisState, ": `thisFile'"
  5.     import excel "$rawPop2000/`thisFile'", /// 
>     sheet("ST-EST00INT-02") /// 
>     cellrange(A4:N5) /// 
>     firstrow ///
>     clear
  6.     
.     /*  Drop columns */
.     drop A B M N
  7.     
.     /* State ID */
.     generate state = thisState
  8.     
.     /* Rename columns to prep for reshape */
.     scalar yearCount = 2000
  9.         foreach var of varlist C-L{
 10.             local j: display int(yearCount)
 11.             rename `var' pop`j'
 12.             scalar yearCount = yearCount + 1
 13.         }
 14.     
.     /* 
>         Reshape the data to a long format
>     */
.     reshape long pop, i(state) j(y

(14 vars, 1 obs)
(j = 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations                1   ->   10          
Number of variables                  11   ->   3           
j variable (10 values)                    ->   year
xij variables:
            pop2000 pop2001 ... pop2009   ->   pop
-----------------------------------------------------------------------------
pop already numeric; no replace
file Data/Processed/US_Census/2000-2010/AR.dta saved
>>>Importing data for  WI : WI.xls
(14 vars, 1 obs)
(j = 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations                1   ->   10          
Number of variables                  11   ->   3           
j variable (10 values)              


Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations                1   ->   10          
Number of variables                  11   ->   3           
j variable (10 values)                    ->   year
xij variables:
            pop2000 pop2001 ... pop2009   ->   pop
-----------------------------------------------------------------------------
pop already numeric; no replace
file Data/Processed/US_Census/2000-2010/AL.dta saved
>>>Importing data for  WV : WV.xls
(14 vars, 1 obs)
(j = 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations                1   ->   10          
Number of variables                  11   ->   3           
j variable (10 values)                    ->   year
xij variables:
            pop2000 pop2001 ... pop2009   

In [29]:
%%stata 
/* Append all the State-level Stata files into a single file */   
local fileList: dir "$processedPop2000" files "*"
local n_files: word count `fileList'
 
forval i=1/`n_files' {
    local thisFile `: word `i' of `fileList''
    scalar thisState = substr("`thisFile'", 1, 2)                           
    

    /* Special actions for first file */
    if `i'==1 {
        di ">>>Importing data for ", thisState, ": `thisFile'"
        use "$processedPop2000/`thisFile'", clear
    }

    /* Actions for all subsequent files */
    if `i'!=1 {
        di ">>>Importing data for ", thisState, ": `thisFile'"
        append using "$processedPop2000/`thisFile'"
    }
    
    if `i'==`n_files'{
        save "$mergedPop2000", replace
    }

}


. /* Append all the State-level Stata files into a single file */   
. local fileList: dir "$processedPop2000" files "*"

. local n_files: word count `fileList'

.  
. forval i=1/`n_files' {
  2.     local thisFile `: word `i' of `fileList''
  3.     scalar thisState = substr("`thisFile'", 1, 2)                         
>   
  4.     
. 
.     /* Special actions for first file */
.     if `i'==1 {
  5.         di ">>>Importing data for ", thisState, ": `thisFile'"
  6.         use "$processedPop2000/`thisFile'", clear
  7.     }
  8. 
.     /* Actions for all subsequent files */
.     if `i'!=1 {
  9.         di ">>>Importing data for ", thisState, ": `thisFile'"
 10.         append using "$processedPop2000/`thisFile'"
 11.     }
 12.     
.     if `i'==`n_files'{
 13.         save "$mergedPop2000", replace
 14.     }
 15. 
. }
>>>Importing data for  GA : GA.dta
>>>Importing data for  KS : KS.dta
>>>Importing data for  IL : IL.dta
>>>Importing data for  SC : SC.dta
>>>Importing data f

In [30]:
%%stata
list in 1/10


     +------------------------+
     | state   year       pop |
     |------------------------|
  1. |    GA   2000   8227303 |
  2. |    GA   2001   8377038 |
  3. |    GA   2002   8508256 |
  4. |    GA   2003   8622793 |
  5. |    GA   2004   8769252 |
     |------------------------|
  6. |    GA   2005   8925922 |
  7. |    GA   2006   9155813 |
  8. |    GA   2007   9349988 |
  9. |    GA   2008   9504843 |
 10. |    GA   2009   9620846 |
     +------------------------+


### US Census Population Estimates 2010-2019

In [31]:
%%stata
import excel "$rawPop2010/2010_2019_PopEst.xlsx", ///
    sheet("Data") ///
    cellrange(A2:M54) /// 
    firstrow ///
    clear

/*
    Drop B which is"2010 Census population"
    Drop C which is the estimate base
    Data will only include population estimates 
      July 2010- July 2019
*/
drop Population C


. import excel "$rawPop2010/2010_2019_PopEst.xlsx", ///
>     sheet("Data") ///
>     cellrange(A2:M54) /// 
>     firstrow ///
>     clear
(13 vars, 52 obs)

. 
. /*
>     Drop B which is"2010 Census population"
>     Drop C which is the estimate base
>     Data will only include population estimates 
>       July 2010- July 2019
> */
. drop Population C

. 


In [32]:
%%stata
// Rename each of the population for reshape cmd
scalar i = 2010
foreach var of varlist D-M{
    local j: display int(i)
    rename `var' pop`j'
    scalar i = i + 1
}
rename GeographicAreaName stateName


. // Rename each of the population for reshape cmd
. scalar i = 2010

. foreach var of varlist D-M{
  2.     local j: display int(i)
  3.     rename `var' pop`j'
  4.     scalar i = i + 1
  5. }

. rename GeographicAreaName stateName

. 


In [33]:
%%stata
list in 1
merge 1:1 stateName using "$raw/StateCodes.dta", ///
    keepusing(state)
list in 1


. list in 1

     +----------------------------------------------------------------------+
  1. | stateN~e |   pop2010 |   pop2011 |   pop2012 |   pop2013 |   pop2014 |
     |  Alabama | 4,785,437 | 4,799,069 | 4,815,588 | 4,830,081 | 4,841,799 |
     |----------------------------------------------------------------------|
     |    pop2015  |    pop2016  |    pop2017  |    pop2018  |    pop2019   |
     |  4,852,347  |  4,863,525  |  4,874,486  |  4,887,681  |  4,903,185   |
     +----------------------------------------------------------------------+

. merge 1:1 stateName using "$raw/StateCodes.dta", ///
>     keepusing(state)

    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                                52  (_merge==3)
    -----------------------------------------

. list in 1

     +----------------------------------------------------------------------+
  1. | stateN~e |   pop20

In [34]:
%%stata
drop stateName _merge

In [35]:
%%stata
reshape long pop, i(state) j(year) 

(j = 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations               52   ->   520         
Number of variables                  11   ->   3           
j variable (10 values)                    ->   year
xij variables:
            pop2010 pop2011 ... pop2019   ->   pop
-----------------------------------------------------------------------------


In [36]:
%%stata
list in 1


     +------------------------+
     | state   year       pop |
     |------------------------|
  1. |    AK   2010   713,910 |
     +------------------------+


In [37]:
%%stata
/* 
Convert to Strings by ignoring the ",".  
  Raw data is formated with comma separator for thousands
  and millions.
*/
destring(pop), replace ignore(",")
label variable pop "US Census Population Estimate"


. /* 
> Convert to Strings by ignoring the ",".  
>   Raw data is formated with comma separator for thousands
>   and millions.
> */
. destring(pop), replace ignore(",")
pop: character , removed; replaced as long

. label variable pop "US Census Population Estimate"

. 


---

## Merge 2000-2009 and 2010-2019 Census Data

In [38]:
%%stata
append using "$mergedPop2000"

In [39]:
%%stata
sort state year
summarize
list if state=="MO"
list if state=="CA"


. sort state year

. summarize

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
       state |          0
        year |        880    2010.409    5.697406       2000       2019
         pop |        880     6727482     6993185     564487   3.95e+07

. list if state=="MO"

     +------------------------+
     | state   year       pop |
     |------------------------|
411. |    MO   2000   5607285 |
412. |    MO   2001   5641142 |
413. |    MO   2002   5674825 |
414. |    MO   2003   5709403 |
415. |    MO   2004   5747741 |
     |------------------------|
416. |    MO   2005   5790300 |
417. |    MO   2006   5842704 |
418. |    MO   2007   5887612 |
419. |    MO   2008   5923916 |
420. |    MO   2009   5961088 |
     |------------------------|
421. |    MO   2010   5995974 |
422. |    MO   2011   6010275 |
423. |    MO   2012   6024367 |
424. |    MO   2013   6040715 |
425. |    MO   2014  

---

## Generate Calculated Parameters

### Add ln(pop)

In [40]:
%%stata
generate lnPop=ln(pop)

## Save Final Census Data

In [41]:
%%stata
save $finalCensusData, replace


file Data/Processed/US_Census/MergedCensus.dta saved


In [42]:
%%capture
%%stata
clear
log close